In [3]:
!nvidia-smi

Tue May 11 09:21:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import numpy as np 
import pandas as pd 
from tensorflow.keras.models import *
from tensorflow.keras.layers import *  
from tensorflow.keras.callbacks import *
from tqdm import tqdm
import time
import random
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow_probability import distributions as tfd


In [2]:
## season 2 dataframes 
train_x_2 = pd.read_csv('./drive/MyDrive/bitTrader2/train_x_df.csv')
train_y_2 = pd.read_csv('./drive/MyDrive/bitTrader2/train_y_df.csv') 
test_x_2 = pd.read_csv('./drive/MyDrive/bitTrader2/test_x_df.csv') 
submission = pd.read_csv('./drive/MyDrive/bitTrader2/sample_submission.csv') 

train_x_2.shape, train_y_2.shape, test_x_2.shape, submission.shape


((10572180, 12), (919320, 12), (738300, 12), (535, 3))

In [4]:
def df2d_to_array3d(df_2d):
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    return array_3d


x_train = df2d_to_array3d(train_x_2) 
y_train = df2d_to_array3d(train_y_2) 
x_test = df2d_to_array3d(test_x_2) 

x_train.shape, y_train.shape, x_test.shape


((7661, 1380, 10), (7661, 120, 10), (535, 1380, 10))

In [6]:
def plot_series(x_series, y_series, y_predicted):
    #입력 series와 출력 series를 연속적으로 연결하여 시각적으로 보여주는 코드 입니다.
    plt.plot(x_series, label = 'input_series')
    plt.plot(np.arange(len(x_series), len(x_series)+len(y_series)),
             y_series, label = 'actual_series') 
    plt.plot(np.arange(len(x_series), len(x_series)+len(y_predicted)),
             y_predicted, label = 'predicted_series') 
    #plt.axhline(1, c = 'red')
    plt.legend()


In [7]:
def plot_predicted_series(x_series, y_predicted):
    #입력 series와 출력 series를 연속적으로 연결하여 시각적으로 보여주는 코드 입니다.
    plt.plot(x_series, label = 'input_series')
    plt.plot(np.arange(len(x_series), len(x_series)+len(y_predicted)),
             y_predicted, label = 'predicted_series') 
    #plt.axhline(1, c = 'red')
    plt.legend()


In [8]:
full_df = np.concatenate([x_train, y_train], axis = 1) 
full_df.shape

(7661, 1500, 10)

In [97]:
price_data = full_df[:100,:,[1,2,3,4]] 
volume_data = full_df[:100,:,[5,6,7,8,9]] 
id_data = full_df[:100,0,0] 

price_data.shape, volume_data.shape, id_data.shape

((100, 1500, 4), (100, 1500, 5), (100,))

In [98]:
id_data

array([9., 9., 4., 0., 7., 4., 6., 0., 0., 9., 4., 6., 4., 4., 0., 7., 8.,
       9., 9., 5., 1., 9., 6., 4., 9., 5., 4., 9., 9., 7., 0., 4., 4., 6.,
       0., 7., 6., 9., 0., 6., 6., 8., 7., 8., 4., 5., 4., 7., 8., 7., 6.,
       4., 6., 8., 4., 7., 9., 0., 6., 0., 3., 4., 0., 8., 0., 6., 7., 1.,
       4., 7., 8., 6., 4., 5., 6., 6., 5., 7., 0., 8., 8., 9., 9., 6., 1.,
       5., 7., 6., 9., 0., 7., 0., 4., 7., 9., 1., 5., 5., 8., 0.])

In [99]:
# min max scale volume data 
for i in tqdm(range(volume_data.shape[0])): 
  for j in range(volume_data.shape[2]): 
    min_val = np.min(volume_data[i,:,j]) 
    max_val = np.max(volume_data[i,:,j]) 
    volume_data[i,:,j] = (volume_data[i,:,j] - min_val) / (max_val - min_val)

100%|██████████| 100/100 [00:00<00:00, 8994.28it/s]


In [101]:
K = 120 
N = 30  
seq_len = 1500 
features_price = 4 
features_volume = 5 
features_id = 1 
X_price = np.zeros(((seq_len-N-K)*price_data.shape[0],N,features_price)) 
X_volume = np.zeros(((seq_len-N-K)*price_data.shape[0],N,features_volume)) 
X_id = np.zeros(((seq_len-N-K)*price_data.shape[0]))
Y = np.zeros(((seq_len-N-K)*price_data.shape[0]))


cnt = 0 
for j in tqdm(range(price_data.shape[0]), position = 0, leave = True): 
  i = 0 
  while i+N+K < seq_len: 
    X_price[cnt,:,:] = price_data[j,i:i+N,:]
    X_volume[cnt,:,:] = volume_data[j,i:i+N,:] 
    X_id[cnt] = id_data[j]
    Y[cnt] = price_data[j,i+N+K,0] 
    i += 1 
    cnt += 1 

X_price.shape, X_volume.shape, X_id.shape, Y.shape

100%|██████████| 100/100 [00:00<00:00, 218.36it/s]


((135000, 30, 4), (135000, 30, 5), (135000,), (135000,))

In [68]:
X_id.shape

(135000,)

In [21]:
class Time2Vector(Layer):
    def __init__(self, seq_len, **kwargs):
        super(Time2Vector, self).__init__()
        self.seq_len = seq_len

    def build(self, input_shape):
        '''Initialize weights and biases with shape (batch, seq_len)'''
        self.weights_linear = self.add_weight(name='weight_linear',
                                              shape=(int(self.seq_len),),
                                              initializer='glorot_uniform',
                                              trainable=True)
    
        self.bias_linear = self.add_weight(name='bias_linear',
                                           shape=(int(self.seq_len),),
                                           initializer='glorot_uniform',
                                           trainable=True)
    
        self.weights_periodic = self.add_weight(name='weight_periodic',
                                                shape=(int(self.seq_len),),
                                                initializer='glorot_uniform',
                                                trainable=True)

        self.bias_periodic = self.add_weight(name='bias_periodic',
                                             shape=(int(self.seq_len),),
                                             initializer='glorot_uniform',
                                             trainable=True)

    def call(self, x):
        '''Calculate linear and periodic time features'''
        x = tf.math.reduce_mean(x[:,:,:], axis=-1) 
        time_linear = self.weights_linear * x + self.bias_linear # Linear time feature
        time_linear = tf.expand_dims(time_linear, axis=-1) # Add dimension (batch, seq_len, 1)
    
        time_periodic = tf.math.sin(tf.multiply(x, self.weights_periodic) + self.bias_periodic)
        time_periodic = tf.expand_dims(time_periodic, axis=-1) # Add dimension (batch, seq_len, 1)
        return tf.concat([time_linear, time_periodic], axis=-1) # shape = (batch, seq_len, 2)
   
    def get_config(self): # Needed for saving and loading model with custom layer
        config = super().get_config().copy()
        config.update({'seq_len': self.seq_len})
        return config


In [22]:
def positional_encoding(max_position, d_model, min_freq=1e-4):
    position = tf.range(max_position, dtype=tf.float32)
    mask = tf.range(d_model)
    sin_mask = tf.cast(mask%2, tf.float32)
    cos_mask = 1-sin_mask
    exponent = 2*(mask//2)
    exponent = tf.cast(exponent, tf.float32)/tf.cast(d_model, tf.float32)
    freqs = min_freq**exponent
    angles = tf.einsum('i,j->ij', position, freqs)
    pos_enc = tf.math.cos(angles)*cos_mask + tf.math.sin(angles)*sin_mask
    return pos_enc


In [117]:
epsilon = 1e-10 # some very small number 

def transformer_block(inputs, node, drop_rate, activation): 
    attn_output = MultiHeadAttention(num_heads = 4, key_dim = node)(inputs, inputs) 
    attn_output = Dropout(drop_rate)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(inputs + attn_output)
    ffn_output = Dense(node, activation = activation)(out1)
    ffn_output = Dense(node)(ffn_output)
    ffn_output = Dropout(drop_rate)(ffn_output)
    out2 = LayerNormalization(epsilon=1e-6)(out1 + ffn_output) 
    return out2 

def transformer_model(inputs, id_inputs, node=64, activation='relu', drop_rate=0.2, num_layers=3): 
    time_embedding = Time2Vector(N)
    bn = BatchNormalization()(inputs)
    x = time_embedding(bn)
    x = Concatenate()([bn, x])
    x = Conv1D(node, 5, activation=activation, padding='same')(x)
    x = MaxPooling1D(3)(x) 
    x = Conv1D(node*3, 5, activation=activation, padding='same')(x)
    x = MaxPooling1D(3)(x)
    x = Dropout(drop_rate)(x)
    positions = positional_encoding(x.shape[1], node*3)
    id_info = Embedding(input_dim = 10, output_dim = node*3)(id_inputs)
    x = x + positions + id_info    
    for i in range(num_layers): 
        x = transformer_block(x, node*3, drop_rate, activation) 
        x = BatchNormalization()(x) 
    x = GlobalAveragePooling1D()(x) 
    return x

def build_model(node=64, activation='relu', drop_rate=0.2, num_layers=3): 
    price_inputs = Input((N, features_price))
    volume_inputs = Input((N, features_volume)) 
    id_inputs = Input((features_id))

    x_p = transformer_model(price_inputs, id_inputs) 
    x_v = transformer_model(volume_inputs, id_inputs)

    x = Concatenate()([x_p, x_v])
    x = Dense(x_p.shape[-1], activation = 'relu')(x) 
    x = BatchNormalization()(x) 
    x = Dense(x_p.shape[-1]/3, activation = 'relu')(x) 
    x = BatchNormalization()(x) 
    x = Dense(1, activation = 'relu')(x) 
    model = Model(inputs=[price_inputs, volume_inputs, id_inputs], outputs=x) 
    model.compile(loss='mape',optimizer='adam',metrics=['mae','mse','mape'])
    return model

In [115]:
model = build_model()

In [116]:
model_path = 'transformer_price_volume_epoch_{epoch:03d}_val_{val_loss:.3f}.h5'
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 3, verbose = 1, factor = 0.5)
checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 10) 


history = model.fit([X_price, X_volume, X_id], 
                     Y, 
                     batch_size = 16, 
                     epochs = 50, 
                     callbacks = [learning_rate_reduction, checkpoint], 
                     validation_split = 0.1)

Epoch 1/50
 520/7594 [=>............................] - ETA: 2:10 - loss: 100.0000 - mae: 1.0018 - mse: 1.0051 - mape: 100.0000

KeyboardInterrupt: ignored

# Make Prediction